In [1]:
pip install --user --upgrade google-cloud-language

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import xml.etree.ElementTree as ET
from google.cloud import language_v1 as language
import pandas as pd
#Print all columns and all rows in a panda dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [27]:
resp = requests.get('https://www.db.com/api/content/limit/15/offset/0/render/false/type/xml/query/+(categories:adHocRelease%20categories:event1%20categories:mediaRelease%20categories:news%20categories:research)%20+C03News.publishDate:[20210101000000%20to%2020211231235959]%20+((languageId:1))%20+deleted:false')

In [28]:
tree = ET.ElementTree(ET.fromstring(resp.content))

In [29]:
root = tree.getroot()

In [30]:
newsitems = []

In [41]:
for item in root.findall('./'):
    for child in item:
        if child.tag == 'richText' and len(child.text.split(' ')) > 20:
            newsitems.append(child.text)

In [42]:
def gcp_classify_text(text):
    client = language.LanguageServiceClient()
    document = language.Document(content=text, type_=language.Document.Type.PLAIN_TEXT)

    response = client.classify_text(document=document)

    for category in response.categories:
        print("=" * 80)
        print(f"category  : {category.name}")
        print(f"confidence: {category.confidence:.0%}")

In [43]:
for item in newsitems:
    gcp_classify_text(item)

InvalidArgument: 400 Invalid text content: too few tokens (words) to process.